In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import expipe
from expipe import Browser
import pandas as pd
import numpy as np
import sys
import exdir

sys.path.append('../ca2-mec')
import data_processing as dp

In [3]:
project_path = dp.project_path()
#project_path = '../../CA2prosjektmappe'
project = expipe.get_project(project_path)
actions = project.actions

In [4]:
dataframes = actions['dataframes']
sessions = pd.read_csv(dataframes.data_path('sessions'))
sessions.head()
print(len(sessions), len(np.unique(sessions.action)))

5 5


In [5]:
def transform_coordinates(x, y, theta=90, **kwargs):
    """
    Transform tracking coordinates to align with physical coordinates
    For this project (CA2 MEC): rotate recorded coordinates 90 degrees, 
    followed by a shift to make values positive afterwards.
    """
    # rotate x,y coordinates 90 degrees using a 2D rotation matrix transform
    theta = np.radians(theta)
    r = np.array(( (np.cos(theta), -np.sin(theta)),
                   (np.sin(theta),  np.cos(theta)) ))
    coords = r @ np.array([x,y])
    # shift new x-coordinates to be positive
    coords -= np.array([[-1],[0]])
    return coords

def corner_masks(x, y, margin=0.4, **kwargs):
    """
    Find corner and center of box masks.
    """
    assert margin < 0.5 and margin > 0, "OBS! Margin must be positive and max half the box size, i.e. 0.5 OBS!"
    
    # center x and y coordinates
    x = x - 0.5
    y = y - 0.5
    pos = np.array([x, y]).T
    
    # create cardinal basis vectors
    ex, ey = np.arange(2), np.arange(2)[::-1]
    
    # create corner vectors: TR, TL, BL and BR, respectively
    corners = np.array([ex+ey, -ex+ey, -ex-ey, ex-ey])
    
    corner_masks = np.zeros((len(x), 5),dtype=bool)
    for i, corner in enumerate(corners):
        # find which quadrant positions are in, and if they are within the
        # box margins. The intersection gives the grand mask.
        quadrant_mask = ((pos * corner) > 0).all(axis=-1)
        margin_mask = (np.abs(pos) > (np.ones(2) * (0.5 - margin))).all(axis=-1)
        corner_masks[:,i] = quadrant_mask & margin_mask
    
    # inverse of union over corner masks
    corner_masks[:,-1] = ~np.array(corner_masks).any(axis=-1)
    return corner_masks

def social_label(action_str):
    """
    socializing can happen at either of the four corners of the box (space).
    there are four types of socializing: nobox (-1) empty (0), familiar (1) or novel (2).
    this gives a 4d-vector with four possible categories each.
    """
    
    def social_category(str1, str2):
        """ helper function """
        if str1 == 'nobox':
            return -1
        if str2 == 'e':
            return 0
        if str2 == 'f':
            return 1
        if str2 == 'n':
            return 2
    
    tags = project.require_action(action_str).attributes['tags']
    social_types = {'s': np.zeros(4), 'o': np.zeros(4)}
    for tag in tags:
        if not 'corner' in tag:
            continue
        stag = tag.split('_')[1:]
        # TR, TL, BL and BR, respectively
        if stag[1] == 'tr':
            social_types[stag[0]][0] = social_category(*stag[-2:])
        elif stag[1] == 'tl':
            social_types[stag[0]][1] = social_category(*stag[-2:])
        elif stag[1] == 'bl':
            social_types[stag[0]][2] = social_category(*stag[-2:])
        elif stag[1] == 'br':
            social_types[stag[0]][3] = social_category(*stag[-2:])
    
    return social_types

In [6]:
def process(data_loader: dp.Data, t_start=0.0, t_stop=None, margin=0.4):
    """Comments"""
    def process_row(row) -> None:
        action_id = row['action']
        x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v'])

        # Choose tracking data time interval (IN SECONDS :D)
        if t_start is not None and t_stop is not None:
            mask = (t < t_stop) & (t > t_start)
            x, y, t = x[mask], y[mask], t[mask]
            
        total_recording_time = t[-1]        
        x, y = transform_coordinates(x, y)
        social_types = social_label(action_id)['s']
        box_locations = ~(social_types== -1) # True where there is 'box', false on 'nobox'
        box_idx_masks = np.append(box_locations, False)
        cms = corner_masks(x=x, y=y, margin=margin)
        sm1,sm2 = cms[:,box_idx_masks].T
        # nsm = np.sum(cms[:,~box_idx_masks], axis=-1).astype(bool) # union of rest of space
        
        samp_freq = len(t) / t[-1]
        top_right_time = np.sum(sm1) / samp_freq
        bottom_left_time = np.sum(sm2) / samp_freq
        sdi = (top_right_time - bottom_left_time) / (top_right_time + bottom_left_time)
        
        # Save values
        row['total-time-in-seconds'] = t[-1]
        row['time-spent-in-top-social-zone'] = top_right_time
        row['time-spent-in-bottom-social-zone'] = bottom_left_time
        row['social-discrimination-index'] = sdi
        print(row)
        
    return process_row

In [7]:
max_speed = 1 # m/s only used for speed score
min_speed = 0.02 # m/s only used for speed score
position_sampling_rate = 100 # for interpolation
position_low_pass_frequency = 6 # for low pass filtering of position

box_size = [1.0, 1.0]
bin_size=0.01
#smoothing = 0.05
baseline_duration = None

data_loader = dp.Data(
    position_sampling_rate=position_sampling_rate, 
    position_low_pass_frequency=position_low_pass_frequency,
    box_size=box_size, bin_size=bin_size, stim_mask=False, baseline_duration=baseline_duration,
)

social_margin = 0.4
t_start,t_stop = 0.0,300

sessions.apply(process(data_loader,t_start=t_start,t_stop=t_stop, margin=social_margin), axis=1)

action                              144-100621-1
total-time-in-seconds                    299.991
time-spent-in-top-social-zone            19.7101
time-spent-in-bottom-social-zone           14.61
social-discrimination-index             0.148601
Name: 0, dtype: object
action                              144-100621-2
total-time-in-seconds                    299.997
time-spent-in-top-social-zone            101.659
time-spent-in-bottom-social-zone         27.0697
social-discrimination-index              0.57943
Name: 1, dtype: object
action                              144-100621-3
total-time-in-seconds                    299.994
time-spent-in-top-social-zone            63.4688
time-spent-in-bottom-social-zone         42.7592
social-discrimination-index             0.194954
Name: 2, dtype: object
action                              144-100621-4
total-time-in-seconds                    299.999
time-spent-in-top-social-zone            99.6797
time-spent-in-bottom-social-zone         46.9098


0    None
1    None
2    None
3    None
4    None
dtype: object

In [8]:
sessions.to_csv(dataframes.data_path('sessions'), index=False)
